In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import warnings
import os
from numbers import Integral

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from segmentation import *
from util import *
from matriarch_stub import *

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(20)

## Run

In [ ]:
funcs = {"mean": np.mean, "median": np.median}
#'p0.05': partial(np.percentile, q=5),
#'p0.20': partial(np.percentile, q=20),
#'p0.70': partial(np.percentile, q=70),
#'p0.95': partial(np.percentile, q=95)}

In [ ]:
base_filename = "/n/scratch2/jqs1/fidelity/190313"
fluorescence_filenames = glob(os.path.join(base_filename, "fluorescence/*.nd2"))
phase_filenames = glob(os.path.join(base_filename, "phase/*_0001.nd2"))
sandwich_filenames = glob(os.path.join(base_filename, "sandwich/*_0001.nd2"))

In [ ]:
dark_frames = nd2_to_dask(
    os.path.join(base_filename, "calibration/dark_100ms.nd2"), 0, "Dark"
)
dark_frame = dark_frames.mean(axis=0)
# dark_std = dark_frames.std(axis=0)

In [ ]:
a = client.compute(dark_frame)

In [ ]:
del a

In [ ]:
client.restart()

In [ ]:
client.persist?

In [ ]:
a

In [ ]:
# gfp_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_FITC_cfpslide_100ms_20pct.nd2').median(axis=0)
# mcherry_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_gfpslide_100ms_000.nd2').median(axis=0)
# mcherry_flat_lowpower = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_30ms_0.35pct_laser.nd2').median(axis=0)

In [ ]:
flats = {"FITC": fitc_flat, "MCHERRY": mcherry_flat}

In [ ]:
data_futures = {}
for photobleaching_filename in fluorescence_filenames[:]:
    data_futures[photobleaching_filename] = process_file(
        funcs,
        photobleaching_filename,
    )

for photobleaching_filename in phase_filenames[:]:
    segmentation_filename = photobleaching_filename.replace("_0001.nd2", ".nd2")
    data_futures[segmentation_filename] = process_file(
        funcs, photobleaching_filename, segmentation_filename=segmentation_filename
    )

for initial_filename in sandwich_filenames[:]:
    segmentation_filename = initial_filename.replace("_0001.nd2", ".nd2")
    photobleaching_filename = initial_filename.replace("_0001.nd2", "_0002.nd2")
    final_filename = initial_filename.replace("_0001.nd2", "_0003.nd2")
    data_futures[segmentation_filename] = process_file(
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        initial_filename=initial_filename,
        final_filename=final_filename,
    )

In [ ]:
a = client.compute(data_futures)

In [ ]:
a

In [ ]:
client.gather(a)

In [ ]:
data_futures[
    "/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_50pct_laser.nd2"
][0]["regionprops"].compute()

In [ ]:
data_futures[
    "/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_50pct_laser.nd2"
][0]["segmentation_frame"].compute()

## Save data

In [ ]:
data = client.gather(data_futures)

In [ ]:
with open("190313photobleaching_flatcorr.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# time: 96593s
# 65% map_over_labels, 30% get_nd2_frame
# time: 109885s (3/22)
# 56% map_over_labels 43% get_nd2_frame

In [ ]:
%%output size=200
hv.HoloMap(
    {
        filename: RevImage(permute_labels(data[filename][0]["labels"]))
        for filename in data.keys()
    }
)

# Plotting

In [ ]:
with open("190311photobleaching.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
list(trace_res.keys())

In [ ]:
b = trace_res["/n/scratch2/jqs1/fidelity/190311/190311_205a_10ms_laser10pct_001.nd2"]

In [ ]:
plt.plot(np.log(b[0]["mean"][10]))

In [ ]:
b[1].hist("area", bins=100)

In [ ]:
traces = z["/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"][0][
    "traces"
]["mean"]

In [ ]:
traces = b[0]["mean"]

In [ ]:
idxs = np.random.permutation(traces.shape[0])

In [ ]:
%%output size=250
curves = [
    {
        "x": np.arange(traces.shape[1]),
        "y": np.log(traces[i] / traces[i, 0]),
        "i": idxs[i],
    }
    for i in range(traces.shape[0] // 10)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20")

In [ ]:
%%output size=250
hv.Path((np.arange(traces.shape[1]), traces.T))

In [ ]:
%%output size=250
hv.Overlay.from_values([hv.Curve(t) for t in traces])

In [ ]:
%%output size=250
hv.Image(img) + hv.Image(segment(img))